In [1]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [2]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [3]:
file = 'data/ohlcv_NEARUSDT_1h_from_1602651600000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2020-10-14 05:00:00,1.3000,1.8800,1.1111,1.3286,9144858.53,2020-10-14 05:59:59.999000064,18549.208333
1,2020-10-14 06:00:00,1.3292,1.3385,1.1251,1.1457,3802846.21,2020-10-14 06:59:59.999000064,18549.250000
2,2020-10-14 07:00:00,1.1588,1.3099,1.1575,1.2517,1802968.37,2020-10-14 07:59:59.999000064,18549.291667
3,2020-10-14 08:00:00,1.2597,1.3199,1.1760,1.2077,1320731.01,2020-10-14 08:59:59.999000064,18549.333333
4,2020-10-14 09:00:00,1.2087,1.2561,1.1638,1.1782,1020106.92,2020-10-14 09:59:59.999000064,18549.375000
...,...,...,...,...,...,...,...,...
13549,2022-05-02 12:00:00,11.1570,11.3510,11.1260,11.1580,600371.50,2022-05-02 12:59:59.999000064,19114.500000
13550,2022-05-02 13:00:00,11.1610,11.5550,11.1000,11.4870,1502353.10,2022-05-02 13:59:59.999000064,19114.541667
13551,2022-05-02 14:00:00,11.4870,12.0210,11.2730,11.9240,2185969.40,2022-05-02 14:59:59.999000064,19114.583333
13552,2022-05-02 15:00:00,11.9240,12.0270,11.5910,11.6090,1052520.10,2022-05-02 15:59:59.999000064,19114.625000


In [4]:
from app.indicators import find_swings
data_ = find_swings(data)

In [5]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [6]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,1.8800,1.1111,1.8800,0.0,0,False,False,0.000,0.000,0.000,0.00,False,False,False,False
1,1.3385,1.1251,1.3385,NaN,0,False,False,0.000,0.000,0.000,0.00,False,False,False,False
2,1.8800,0.0000,1.8800,-1.0,0,False,False,0.000,0.000,0.000,0.00,False,False,False,False
3,1.8800,0.0000,1.8800,-1.0,1,False,True,0.000,1.880,0.000,1.88,False,False,True,False
4,1.8800,0.0000,1.8800,-1.0,2,False,True,0.000,1.880,0.000,0.00,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13549,12.1990,11.6860,12.1990,-1.0,3,False,True,0.000,12.199,0.000,0.00,False,False,False,False
13550,12.1990,11.0870,11.0870,1.0,0,False,False,0.000,0.000,0.000,0.00,False,False,False,False
13551,12.1990,11.0870,11.0870,1.0,1,True,False,11.087,0.000,11.087,0.00,True,False,False,False
13552,12.1990,11.0870,11.0870,1.0,2,True,False,11.087,0.000,0.000,0.00,False,False,False,False


In [7]:
data__=data__*1

In [8]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,1.8800,1.1111,1.8800,0.0,0,0,0,0.000,0.000,0.000,0.00,0,0,0,0
1,1.3385,1.1251,1.3385,NaN,0,0,0,0.000,0.000,0.000,0.00,0,0,0,0
2,1.8800,0.0000,1.8800,-1.0,0,0,0,0.000,0.000,0.000,0.00,0,0,0,0
3,1.8800,0.0000,1.8800,-1.0,1,0,1,0.000,1.880,0.000,1.88,0,0,1,0
4,1.8800,0.0000,1.8800,-1.0,2,0,1,0.000,1.880,0.000,0.00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13549,12.1990,11.6860,12.1990,-1.0,3,0,1,0.000,12.199,0.000,0.00,0,0,0,0
13550,12.1990,11.0870,11.0870,1.0,0,0,0,0.000,0.000,0.000,0.00,0,0,0,0
13551,12.1990,11.0870,11.0870,1.0,1,1,0,11.087,0.000,11.087,0.00,1,0,0,0
13552,12.1990,11.0870,11.0870,1.0,2,1,0,11.087,0.000,0.000,0.00,0,0,0,0


In [9]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [10]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [11]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2020-10-14 05:00:00,1.3000,1.8800,1.1111,1.3286,9144858.53,0,0,0,0
1,2020-10-14 06:00:00,1.3292,1.3385,1.1251,1.1457,3802846.21,0,0,0,0
2,2020-10-14 07:00:00,1.1588,1.3099,1.1575,1.2517,1802968.37,0,0,0,0
3,2020-10-14 08:00:00,1.2597,1.3199,1.1760,1.2077,1320731.01,0,0,1,0
4,2020-10-14 09:00:00,1.2087,1.2561,1.1638,1.1782,1020106.92,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
13549,2022-05-02 12:00:00,11.1570,11.3510,11.1260,11.1580,600371.50,0,0,0,0
13550,2022-05-02 13:00:00,11.1610,11.5550,11.1000,11.4870,1502353.10,0,0,0,0
13551,2022-05-02 14:00:00,11.4870,12.0210,11.2730,11.9240,2185969.40,1,0,0,0
13552,2022-05-02 15:00:00,11.9240,12.0270,11.5910,11.6090,1052520.10,0,0,0,0


In [12]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [13]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2020-10-14 05:00:00,1.3000,1.8800,1.1111,1.3286,9144858.53,0,0,0,0,1
1,2020-10-14 06:00:00,1.3292,1.3385,1.1251,1.1457,3802846.21,0,0,0,0,1
2,2020-10-14 07:00:00,1.1588,1.3099,1.1575,1.2517,1802968.37,0,0,0,0,1
3,2020-10-14 08:00:00,1.2597,1.3199,1.1760,1.2077,1320731.01,0,0,1,0,0
4,2020-10-14 09:00:00,1.2087,1.2561,1.1638,1.1782,1020106.92,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
13549,2022-05-02 12:00:00,11.1570,11.3510,11.1260,11.1580,600371.50,0,0,0,0,1
13550,2022-05-02 13:00:00,11.1610,11.5550,11.1000,11.4870,1502353.10,0,0,0,0,1
13551,2022-05-02 14:00:00,11.4870,12.0210,11.2730,11.9240,2185969.40,1,0,0,0,0
13552,2022-05-02 15:00:00,11.9240,12.0270,11.5910,11.6090,1052520.10,0,0,0,0,1


In [14]:
df.set_index('o_ts',inplace=True)

In [15]:
df.to_csv('data/transformed_data/NEARUSDT_1h_transformed.csv')

In [16]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     10415
    1   0   0   0       899
    0   0   1   0       884
1   0   0   0   0       697
0   0   1   0   0       659
dtype: int64

In [17]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [18]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [19]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [20]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [21]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [22]:
X, y = get_X_y(df, 1000, 21)
print(X.shape)
print(y.shape)

(1000, 20, 10)
(1000, 5)


In [23]:
assert np.isnan(X).sum() == 0

In [24]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [25]:
X_train, y_train = get_X_y(data_train, 1000, 21)
X_test, y_test = get_X_y(data_test, 100, 21)

In [26]:
assert X_train.shape == (1000, 20, 10)

In [27]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [28]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [29]:
model = init_model()
model.summary()

2022-05-06 13:11:15.482083: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 13:11:15.510610: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [31]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [32]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/75
13/13 [==============================] - 2s 48ms/step - loss: 2.9653 - accuracy: 0.6500 - val_loss: 1.1197 - val_accuracy: 0.7500
Epoch 2/75
13/13 [==============================] - 0s 12ms/step - loss: 2.6617 - accuracy: 0.6988 - val_loss: 1.1141 - val_accuracy: 0.5750
Epoch 3/75
13/13 [==============================] - 0s 12ms/step - loss: 2.4737 - accuracy: 0.5612 - val_loss: 1.0588 - val_accuracy: 0.5350
Epoch 4/75
13/13 [==============================] - 0s 12ms/step - loss: 2.2674 - accuracy: 0.5450 - val_loss: 0.9508 - val_accuracy: 0.6000
Epoch 5/75
13/13 [==============================] - 0s 12ms/step - loss: 2.2267 - accuracy: 0.5213 - val_loss: 0.9632 - val_accuracy: 0.5500
Epoch 6/75
13/13 [==============================] - 0s 11ms/step - loss: 2.0901 - accuracy: 0.5500 - val_loss: 0.9514 - val_accuracy: 0.5200
Epoch 7/75
13/13

In [33]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.8851 - accuracy: 0.6400


[0.8850511312484741, 0.6399999856948853]

In [34]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.34, 0.08, 0.06, 0.26, 0.26],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.1 , 0.  , 0.  , 0.89],
       [0.  , 0.4 , 0.11, 0.02, 0.47],
       [0.  , 0.  , 0.  , 0.61, 0.39],
       [0.  , 0.01, 0.  , 0.  , 0.99],
       [0.  , 0.  , 0.01, 0.15, 0.84],
       [0.08, 0.32, 0.16, 0.19, 0.25],
       [0.14, 0.19, 0.17, 0.16, 0.34],
       [0.02, 0.36, 0.26, 0.07, 0.29],
       [0.33, 0.05, 0.04, 0.18, 0.41],
       [0.13, 0.33, 0.18, 0.14, 0.23],
       [0.33, 0.07, 0.03, 0.11, 0.47],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.01, 0.23, 0.02, 0.38, 0.36],
       [0.14, 0.06, 0.01, 0.04, 0.75],
       [0.17, 0.16, 0.23, 0.13, 0.31],
       [0.02, 0.15, 0.09, 0.03, 0.71],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.02, 0.1 , 0.09, 0.01, 0.78],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.01, 0.32, 0.04, 0.26, 0.37],
       [0.09, 0.16, 0.02, 0.01, 0.72],
       [0.  , 0.  , 0.  ,

In [35]:
model.save('./models/RNN_NEARUSDT_1h.h5')